<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/valvesCvCg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! apt update -q
! apt-get install -q openjdk-25-jdk-headless -q
!pip install neqsim -q
from neqsim import jneqsim
import neqsim
from neqsim.thermo.thermoTools import fluid
from neqsim.process import clearProcess, stream, valve, runProcess
import math

# Valve Cv and Cg simulation
This notebook shows how control valves are simulated using the [NeqSim](https://github.com/equinor/neqsim) process framework. The valve model is implemented in the [`neqsim.processSimulation.processEquipment.valve`](https://github.com/equinor/neqsim/tree/master/src/main/java/neqsim/processSimulation/processEquipment/valve) package as the `ThrottlingValve` class.

## Basic valve simulation
Create a simple gas stream and let it flow through a valve. The downstream pressure is fixed using `setOutletPressure`. When a `Cv` value is specified the mass flow through the valve is calculated from the pressure drop.

In [ ]:
inletTemperature = 20.0  # C
inletPressure = 100.0  # bara
outletPressure = 50.0  # bara

fluid1 = fluid('srk')
fluid1.addComponent('methane', 1.0)
fluid1.setTemperature(inletTemperature, 'C')
fluid1.setPressure(inletPressure, 'bara')
fluid1.setTotalFlowRate(1000.0, 'kg/hr')

clearProcess()
stream1 = stream(fluid1)
valve1 = valve(stream1)
valve1.setOutletPressure(outletPressure, 'bara')
valve1.setCv(80.0)
runProcess()

print('Valve outlet pressure', valve1.getOutStream().getPressure('bara'), 'bara')
print('Valve mass flow', valve1.getOutStream().getFlowRate('kg/hr'))

### Cv and Cg
The valve can also be specified using the gas sizing coefficient `Cg`. Setting one coefficient automatically updates the other according to the standard valve sizing correlation used in NeqSim.

In [ ]:
valve1.setCg(95.0)
print('Updated Cv from Cg:', valve1.getCv())
print('Cg now:', valve1.getCg())

## Valve sizing
The size of a valve is typically given by its flow coefficient. For gas service the coefficients `Cv` or `Cg` are used. For liquid service the metric coefficient `Kv` is often used which is related to `Cv` by `Kv = 0.865 * Cv`.

In [ ]:
Kv = stream1.getFlowRate('m3/hr')/math.sqrt(inletPressure-outletPressure)
Cv_calc = Kv/0.865
print('Calculated Kv', Kv)
print('Corresponding Cv', Cv_calc)

Typical range of Cv values for common valve types (approximate):

| Valve type | Cv range |
|------------|---------|
| 1/2'' globe | 4 - 6 |
| 2'' globe   | 45 - 60 |
| 2'' ball    | 275 - 300 |
| 4'' ball    | 950 - 1300 |

## Valve flow equations
For an incompressible liquid the valve flow coefficient relates flow rate to pressure drop as $$Q = C_v \sqrt{\frac{\Delta P}{G}}$$ where $G$ is the specific gravity.
For compressible gas service NeqSim uses the ANSI/ISA S75.01 correlations. The mass flow is computed from the $C_g$ coefficient by $$W = C_g P_1 \sqrt{\frac{P_2}{P_1} \frac{M}{Z T_1}}$$ where $M$ is molecular weight, $Z$ is compressibility factor and $T_1$ is the upstream temperature.

## Controlling downstream pressure by valve opening
Instead of fixing the outlet pressure it is possible to set a valve opening with `setPercentValveOpening` and let NeqSim calculate the resulting pressure or flow.

In [ ]:
valve1.setOutletPressure(None)
valve1.setPercentValveOpening(50.0)
runProcess()
print('Downstream pressure with 50% opening', valve1.getOutStream().getPressure('bara'))
print('Resulting flow', valve1.getOutStream().getFlowRate('kg/hr'))

### References
- [ISA-75.01 Control Valve Sizing](https://www.isa.org/standards/0100006944)
- [Fisher Control Valve Handbook](https://www.emerson.com/documents/automation/control-valve-handbook-en-3658406.pdf)
- [Control Valve Basics on Wikipedia](https://en.wikipedia.org/wiki/Control_valve)